![AIcrowd-Logo](https://raw.githubusercontent.com/AIcrowd/AIcrowd/master/app/assets/images/misc/aicrowd-horizontal.png)

# Final Solution for [SKELY Challenge](https://www.aicrowd.com/challenges/skely) on AIcrowd
#### Author : Team BlitzCA

## Download Necessary Packages

In [2]:
import sys
!pip install numpy
!pip install pandas
!pip install scikit-learn 
!pip install matplotlib tqdm 

## Download data
The first step is to download the training data and the test data


In [3]:
# #Donwload the datasets
!rm -rf data/
!mkdir data/

!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/skely/v0.1/train.tar.gz -o data/train.tar.gz
!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/skely/v0.1/test.tar.gz -o data/test.tar.gz
!curl https://s3.eu-central-1.wasabisys.com/aicrowd-practice-challenges/public/skely/v0.1/sample_submission.csv -o data/sample_submission.csv
!tar xvzf data/train.tar.gz -C data/
!tar xvzf data/test.tar.gz -C data/


Streaming output truncated to the last 5000 lines.
images/019870.png
images/017685.png
images/011592.png
images/010854.png
images/013385.png
images/015938.png
images/013391.png
images/010698.png
images/011586.png
images/010840.png
images/017849.png
images/019864.png
images/017691.png
images/014398.png
images/012931.png
images/015086.png
images/019694.png
images/017861.png
images/012919.png
images/015910.png
images/010868.png
images/015904.png
images/019680.png
images/017875.png
images/019858.png
images/010129.png
images/018575.png
images/017646.png
images/016558.png
images/011237.png
images/015051.png
images/013420.png
images/013346.png
images/014429.png
images/015737.png
images/012058.png
images/011551.png
images/010897.png
images/018213.png
images/017120.png
images/011545.png
images/019119.png
images/010883.png
images/018207.png
images/017134.png
images/013352.png
images/015723.png
images/015045.png
images/013434.png
images/018561.png
images/017652.png
images/011223.png
images/012702

In [4]:
## Now the data is available at the following locations:

TRAINING_IMAGES_FOLDER = "data/training/images/"
TRAINING_LABELS_PATH = "data/training/labels.csv"
TEST_IMAGES_FOLDER = "data/images"
SAMPLE_SUBMISSION_FILE_PATH = "data/sample_submission.csv"


## Import packages

In [63]:
import os
import tqdm
import cv2

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageOps


## Load Data


In [6]:
os.makedirs("data/whitened train")
os.makedirs("data/whitened test")

In [79]:
training_labels_df = pd.read_csv(TRAINING_LABELS_PATH)

In [50]:
for _idx, row in tqdm.tqdm(training_labels_df.iterrows(), total=training_labels_df.shape[0]):
    filepath = os.path.join(TRAINING_IMAGES_FOLDER,row.filename)
    img = cv2.imread(filepath)

    #== Parameters =======================================================================
    BLUR = 21
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 200
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = (0.0,0.0,1.0) # In BGR format

    img = cv2.imread(filepath)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #-- Edge detection -------------------------------------------------------------------
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)

    #-- Find contours in edges, sort by area ---------------------------------------------
    contour_info = []
    contours, _ = cv2.findContours(gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    for c in contours:
        contour_info.append((
         c,
           cv2.isContourConvex(c),
          cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]

    #-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
    # Mask is black, polygon is white
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))

    mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

    #-- Blend masked img into MASK_COLOR background --------------------------------------
    mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
    img         = img.astype('float32') / 255.0                 #  for easy blending

    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) # Blend
    masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit 


    # split image into channels
    c_red, c_green, c_blue = cv2.split(img)
    c_red_new = np.where(c_red==c_red[0][0], 1, c_red)
    c_green_new = np.where(c_green==c_green[0][0], 1, c_green)
    c_blue_new = np.where(c_blue==c_blue[0][0], 1, c_blue)

    # merge with mask got on one of a previous steps
    img_a = cv2.merge((c_red_new, c_green_new, c_blue_new, mask.astype('float32') / 255.0))


    cv2.imwrite("data/whitened train/"+row.filename, img_a*255)









  0%|          | 0/9999 [00:00<?, ?it/s]







  0%|          | 1/9999 [00:00<1:26:22,  1.93it/s]







  0%|          | 2/9999 [00:01<1:25:24,  1.95it/s]







  0%|          | 3/9999 [00:01<1:28:03,  1.89it/s]







  0%|          | 4/9999 [00:02<1:31:41,  1.82it/s]







  0%|          | 5/9999 [00:02<1:36:00,  1.73it/s]







  0%|          | 6/9999 [00:03<1:38:01,  1.70it/s]







  0%|          | 7/9999 [00:03<1:35:03,  1.75it/s]







  0%|          | 8/9999 [00:04<1:36:06,  1.73it/s]







  0%|          | 9/9999 [00:05<1:38:19,  1.69it/s]







  0%|          | 10/9999 [00:05<1:35:51,  1.74it/s]







  0%|          | 11/9999 [00:06<1:33:17,  1.78it/s]







  0%|          | 12/9999 [00:06<1:35:59,  1.73it/s]







  0%|          | 13/9999 [00:07<1:38:33,  1.69it/s]







  0%|          | 14/9999 [00:08<1:36:21,  1.73it/s]







  0%|          | 15/9999 [00:08<1:33:40,  1.78it/s]







  0%|          | 16/9999 [00:09<1:31:12,  1.82it/s]







  0%|    

KeyboardInterrupt: ignored

In [53]:
test_labels_df = pd.read_csv(SAMPLE_SUBMISSION_FILE_PATH)

for _idx, row in tqdm.tqdm(test_labels_df.iterrows(), total=test_labels_df.shape[0]):
    filepath = os.path.join(TEST_IMAGES_FOLDER,row.filename)
    img = cv2.imread(filepath)

    #== Parameters =======================================================================
    BLUR = 21
    CANNY_THRESH_1 = 10
    CANNY_THRESH_2 = 200
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = (0.0,0.0,1.0) # In BGR format

    img = cv2.imread(filepath)

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    #-- Edge detection -------------------------------------------------------------------
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)

    #-- Find contours in edges, sort by area ---------------------------------------------
    contour_info = []
    contours, _ = cv2.findContours(gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    for c in contours:
        contour_info.append((
         c,
              cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    max_contour = contour_info[0]

    #-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
    # Mask is black, polygon is white
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))

    mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

    #-- Blend masked img into MASK_COLOR background --------------------------------------
    mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
    img         = img.astype('float32') / 255.0                 #  for easy blending

    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) # Blend
    masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit 


    # split image into channels
    c_red, c_green, c_blue = cv2.split(img)
    c_red_new = np.where(c_red==c_red[0][0], 1, c_red)
    c_green_new = np.where(c_green==c_green[0][0], 1, c_green)
    c_blue_new = np.where(c_blue==c_blue[0][0], 1, c_blue)

    # merge with mask got on one of a previous steps
    img_a = cv2.merge((c_red_new, c_green_new, c_blue_new, mask.astype('float32') / 255.0))


    cv2.imwrite("data/whitened test/"+row.filename, img_a*255)










  0%|          | 0/10001 [00:00<?, ?it/s]








  0%|          | 1/10001 [00:00<1:33:35,  1.78it/s]








  0%|          | 2/10001 [00:01<1:37:12,  1.71it/s]








  0%|          | 3/10001 [00:01<1:38:52,  1.69it/s]








  0%|          | 4/10001 [00:02<1:40:06,  1.66it/s]

KeyboardInterrupt: ignored

In [61]:
NEW_TRAINING_IMAGES_FOLDER = "data/whitened train"
NEW_TEST_IMAGES_FOLDER = "/data/whitened test"

In [67]:
for _idx, row in tqdm.tqdm(training_labels_df.iterrows(), total=training_labels_df.shape[0]):
    filepath = os.path.join(
        NEW_TRAINING_IMAGES_FOLDER,
        row.filename
    )
    im = Image.open(filepath)
    im_mirror = ImageOps.mirror(im)
    im_mirror.save(os.path.join(NEW_TRAINING_IMAGES_FOLDER,
        row.filename.split(".")[0]+'_flip.png'))











  0%|          | 0/9999 [00:00<?, ?it/s]









  0%|          | 1/9999 [00:00<57:11,  2.91it/s]









  0%|          | 2/9999 [00:00<56:54,  2.93it/s]









  0%|          | 3/9999 [00:00<55:39,  2.99it/s]









  0%|          | 4/9999 [00:01<54:43,  3.04it/s]









  0%|          | 5/9999 [00:01<54:07,  3.08it/s]









  0%|          | 6/9999 [00:01<55:00,  3.03it/s]









  0%|          | 7/9999 [00:02<54:09,  3.08it/s]









  0%|          | 8/9999 [00:02<53:24,  3.12it/s]









  0%|          | 9/9999 [00:02<53:42,  3.10it/s]









  0%|          | 10/9999 [00:03<52:59,  3.14it/s]









  0%|          | 11/9999 [00:03<57:38,  2.89it/s]


KeyboardInterrupt: ignored

In [80]:
train_flip_df = pd.concat([training_labels_df['xRot'], pd.DataFrame(training_labels_df['filename'].apply(lambda x: x.split(".")[0]+'_flip.png'))], axis=1)

In [81]:
training_labels_df = pd.concat([training_labels_df, train_flip_df], axis=0).reset_index(drop=True)

In [ ]:
def pre_process_data_X(image):
    
    image = image.resize((32,32))
    im_array = np.array(image)
    return im_array
    


ALL_DATA = []

for _idx, row in tqdm.tqdm(training_labels_df.iterrows(), total=training_labels_df.shape[0]):
    filepath = os.path.join(
        NEW_TRAINING_IMAGES_FOLDER,
        row.filename
    )
    im = Image.open(filepath)
    
    data_X = pre_process_data_X(im)
    data_Y = [row.xRot]
    
    ALL_DATA.append((data_X, data_Y))
  



## Split Data into Train and Validation
We split the dataset into Training data and Validation datasets to help us test the generalizability of our models, and to ensure that we are not overfitting on the training set.

In [ ]:
training_set, validation_set= train_test_split(ALL_DATA, test_size=0.2, random_state=42) 

In [ ]:
X_train, y_train = zip(*training_set)
X_val, y_val = zip(*validation_set)
X_train_full, y_train_full = zip(*ALL_DATA)


X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], -1)
X_train_full = np.array(X_train_full)
X_train_full = X_train_full.reshape(X_train_full.shape[0], -1)
y_train = np.array(y_train)
y_train_full = np.array(y_train_full)
X_val = np.array(X_val)
X_val = X_val.reshape(X_val.shape[0], -1)
y_val = np.array(y_val)
print(X_train.shape)
print(y_train.shape)

## Define the Classifier, and Train


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=1, n_jobs=-1, p=1, algorithm='ball_tree')
model.fit(X_train, np.ones(X_train.shape[0]))

## Predict on Validation, and Evaluate

In [ ]:
idxs = model.kneighbors(X_val, return_distance = False)
y_preds = np.zeros((X_val.shape[0], 1))
for i, idx in enumerate(idxs):
    y_preds[i] = y_train[idx]

total_loss = mean_squared_error(y_val, y_preds)
print(total_loss)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1342: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 41129.48898644
Iteration 2, loss = 22749.22529257
Iteration 3, loss = 10433.89035716
Iteration 4, loss = 4334.56568262
Iteration 5, loss = 3627.21814731
Iteration 6, loss = 3497.38697087
Iteration 7, loss = 3410.92176757
Iteration 8, loss = 3350.87261289
Iteration 9, loss = 3307.94813312
Iteration 10, loss = 3277.05173166
Iteration 11, loss = 3255.41088791
Iteration 12, loss = 3238.21565847
Iteration 13, loss = 3226.10013849
Iteration 14, loss = 3215.62099961
Iteration 15, loss = 3209.04319212
Iteration 16, loss = 3204.08637621
Iteration 17, loss = 3199.64622032
Iteration 18, loss = 3194.87411342
Iteration 19, loss = 3192.88077696
Iteration 20, loss = 3188.89238840
Iteration 21, loss = 3185.88886666
Iteration 22, loss = 3182.90967462
Iteration 23, loss = 3180.37219268
Iteration 24, loss = 3177.31085348
Iteration 25, loss = 3175.35553604
Iteration 26, loss = 3174.59038159
Iteration 27, loss = 3171.18820111
Iteration 28, loss = 3171.80883233
Iteration 29, loss = 3168.

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=[10, 10], learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0.5, random_state=None, shuffle=True, solver='adam',
             tol=0.0001, validation_fraction=0.1, verbose=True,
             warm_start=False)

In [ ]:
model_full = KNeighborsRegressor(n_neighbors=1, n_jobs=-1, p=1, algorithm='ball_tree')
model_full.fit(X_train_full, np.ones(X_train_full.shape[0]))

## Load Test Set


In [ ]:
import glob

TEST_DATA = []
TEST_FILENAMES = []

for _test_image_path in tqdm.tqdm(glob.glob(os.path.join(NEW_TEST_IMAGES_FOLDER, "*.png"))):
    filename = os.path.basename(_test_image_path)
    im = Image.open(_test_image_path)
    
    data_X = pre_process_data_X(im)
    TEST_DATA.append(data_X)
    TEST_FILENAMES.append(filename)

100%|██████████| 10001/10001 [24:50<00:00,  6.71it/s]


In [ ]:
TEST_DATA = np.array(TEST_DATA)
TEST_DATA = TEST_DATA.reshape(TEST_DATA.shape[0], -1)


## Make predictions on the test set

In [ ]:
test_predictions = np.zeros((TEST_DATA.shape[0], 1))

test_idxs = model_full.kneighbors(TEST_DATA, return_distance=False)
for i, idx in enumerate(test_idxs):
    test_predictions[i] = y_train_full[idx]



In [ ]:
test_df = pd.DataFrame(test_predictions, columns=['xRot'])
test_df["filename"] = TEST_FILENAMES

## Save the prediction to csv

In [ ]:
test_df.to_csv('submission-Mic 32.csv', index=False)